In [ ]:
# import pandas as pd

# file_path = '/home/fossa/data/dont_worry_about_it/datasets/exoplanets.csv'
# df = pd.read_csv(file_path,  = 'pl_name')

In [ ]:
# 
# 'Walter Weiler', 'Samir Shaker Mahmoud', 'Medhi Lacen'

# df.columns = ['Jersey_Num', 'Position', 'DOB_Age',
#               'Caps', 'Club', 'Country', 'ClubCountry', 'Cup_Year']


# df = df.fillna(0)


# df[df.Caps.isnull()]
# df.fillna(0)

# df = pd.to_datetime(df.Birthday)
df.info()

In [13]:
import pandas as pd

# import a csv from the world wide web
soccer_url = "https://raw.githubusercontent.com/BriChavez/FIFAWorldCup/master/squads.csv"
# read it into our trusty pandas df so we can clean it
df = pd.read_csv(soccer_url)

# set index as player name
df = df.set_index('Player')


In [14]:

# set column names
df.columns = ['Jersey_Num',
              'Position',
              'DOB_Age',
              'Caps',
              'Club',
              'Country',
              'ClubCountry',
              'Cup_Year']
df.describe

<bound method NDFrame.describe of                   Jersey_Num Position                                 DOB_Age  \
Player                                                                          
Ángel Bossio               1      1GK        (1905-05-05)5 May 1905 (aged 25)   
Juan Botasso               1      1GK   (1908-10-23)23 October 1908 (aged 21)   
Roberto Cherro             9      4FW  (1907-02-23)23 February 1907 (aged 23)   
Alberto Chividini          4      2DF  (1907-02-23)23 February 1907 (aged 23)   
Attilio Demaría           10      4FW     (1909-03-19)19 March 1909 (aged 21)   
...                      ...      ...                                     ...   
Park Chu-Young             -      4FW      (1985-07-10)10 July 1985 (aged 28)   
Ji Dong-Won                -      4FW       (1991-05-28)28 May 1991 (aged 23)   
Lee Keun-Ho                -      4FW     (1985-04-11)11 April 1985 (aged 29)   
Koo Ja-Cheol               -      4FW  (1989-02-27)27 February 1989 (aged 2

In [15]:
# replace non integer values, in the column Caps, with 0
df.Caps.str.replace('[^\w\s]', '0')


Player
Ángel Bossio         NaN
Juan Botasso         NaN
Roberto Cherro       NaN
Alberto Chividini    NaN
Attilio Demaría      NaN
                    ... 
Park Chu-Young        62
Ji Dong-Won           26
Lee Keun-Ho           62
Koo Ja-Cheol          35
Kim Shin-Wook         26
Name: Caps, Length: 8897, dtype: object


/home/fossa/deb/tests/dbt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [19]:

# extract words and numbers surrounded by parens. aka 'aged' and age value, in this instance
Age = df.DOB_Age.str.extract(r"\(([A-Za-z0-9 _]+)\)")
# assign the extracted aged info as a column to mess with further
df = df.assign(Age=Age)


<class 'pandas.core.frame.DataFrame'>
Index: 8897 entries, Ángel Bossio to Kim Shin-Wook
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Jersey_Num   8873 non-null   object
 1   Position     8897 non-null   object
 2   DOB_Age      8805 non-null   object
 3   Caps         6086 non-null   object
 4   Club         8897 non-null   object
 5   Country      8897 non-null   object
 6   ClubCountry  8897 non-null   object
 7   Cup_Year     8897 non-null   int64 
 8   Age          8739 non-null   object
dtypes: int64(1), object(8)
memory usage: 695.1+ KB


In [21]:
# reassign our new column the extracted integers from that column, aka just the ages
df = df.assign(Age=lambda x: x['Age'].str.extract('(\d+)'))


<class 'pandas.core.frame.DataFrame'>
Index: 8897 entries, Ángel Bossio to Kim Shin-Wook
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Jersey_Num   8873 non-null   object
 1   Position     8897 non-null   object
 2   DOB_Age      8805 non-null   object
 3   Caps         6086 non-null   object
 4   Club         8897 non-null   object
 5   Country      8897 non-null   object
 6   ClubCountry  8897 non-null   object
 7   Cup_Year     8897 non-null   int64 
 8   Age          8739 non-null   object
dtypes: int64(1), object(8)
memory usage: 695.1+ KB


In [23]:
# reassign as birthday the dob datetime object from inside parens
df = df.assign(Birthday=lambda x: x['DOB_Age'].str.extract(r"\(([0-9 _-]+)\)"))
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8897 entries, Ángel Bossio to Kim Shin-Wook
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Jersey_Num   8873 non-null   object
 1   Position     8897 non-null   object
 2   DOB_Age      8805 non-null   object
 3   Caps         6086 non-null   object
 4   Club         8897 non-null   object
 5   Country      8897 non-null   object
 6   ClubCountry  8897 non-null   object
 7   Cup_Year     8897 non-null   int64 
 8   Age          8739 non-null   object
 9   Birthday     8736 non-null   object
dtypes: int64(1), object(9)
memory usage: 764.6+ KB


In [ ]:
# sorry boys, ya b-day is backward and i dont have time to figure that out
df = df.drop(
    # dropping the backward birthday boys
    index=['Walter Weiler', 'Samir Shaker Mahmoud', 'Medhi Lacen'])



In [27]:
# set birthday as a datetime
df.Birthday = pd.to_datetime(df.Birthday)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8894 entries, Ángel Bossio to Kim Shin-Wook
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Jersey_Num   8870 non-null   object        
 1   Position     8894 non-null   object        
 2   DOB_Age      8802 non-null   object        
 3   Caps         6084 non-null   object        
 4   Club         8894 non-null   object        
 5   Country      8894 non-null   object        
 6   ClubCountry  8894 non-null   object        
 7   Cup_Year     8894 non-null   int64         
 8   Age          8736 non-null   object        
 9   Birthday     8733 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 1022.4+ KB


In [28]:
# fill null numeric values with 0
df = df.fillna(0)
# fill null values from nonnumeric columns with n/a
df = df.fillna('n/a')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8894 entries, Ángel Bossio to Kim Shin-Wook
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Jersey_Num   8894 non-null   object
 1   Position     8894 non-null   object
 2   DOB_Age      8894 non-null   object
 3   Caps         8894 non-null   object
 4   Club         8894 non-null   object
 5   Country      8894 non-null   object
 6   ClubCountry  8894 non-null   object
 7   Cup_Year     8894 non-null   int64 
 8   Age          8894 non-null   object
 9   Birthday     8894 non-null   object
dtypes: int64(1), object(9)
memory usage: 1022.4+ KB


In [3]:









# force my will and turn my age column as an int
df = df.Age.astype(int, errors='ignore')
# force my Caps column to turn into a int column, no matter what it says
df = df.Caps.astype('int', errors='ignore')
# dropping the wordy column as we got what we wanted from it.
df = df.drop(['DOB_Age'], axis = 1)
# check to see we did it right
df.info()


/home/fossa/deb/tests/dbt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


AttributeError: 'Series' object has no attribute 'DOB_Age'

In [ ]:
df.to_csv('data/world_cup.csv')

In [ ]:
# Imports the Google Cloud client library
import os
from google.cloud import storage

# Instantiates a client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dbt-user-creds.json"
client = storage.Client()


# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/world_cup/
bucket = client.get_bucket('world_cup')
# upload the csv to the bucket
blob = bucket.blob('world_cup.csv')
blob.upload_from_filename('data/world_cup.csv')
